In [1]:
from dcicutils import ff_utils
from functions.notebook_functions import *

# get key from file
my_key = get_key('koray_data')

# all usable env names
all_envs = ['data', 'staging', 'fourfront-webdev', 'fourfront-mastertest', 'fourfront-hotseat']
# get admin key from s3
#my_key = ff_utils.get_authentication_with_server({}, ff_env = 'data')

In [2]:
q = '/search/?type=Workflow'
all_wf = ff_utils.search_metadata(q, key=my_key)

In [3]:
print(len(all_wf))

56


In [18]:
qf = '/search/?type=FileFormat'
allowed_formats = [i['file_format'] for i in ff_utils.search_metadata(qf, key=my_key)]
print(len(allowed_formats), 'file formats avaiable')
for i in sorted(allowed_formats):
    print(i)

44 file formats avaiable
bai
bam
bed
beddb
bedpe
bg
bg_px2
bigbed
bw
bwaIndex
chromsizes
clusters
compressed_fasta
compressed_txt
cool
csv
dv
fasta
fastq
flex
hdf5
hic
html
json
juicer_format_restriction_site_file
matrix
mcool
merged_nodups
multires.mv5
nd2
normvector_juicerformat
ome.tiff
other
pairs
pairs_px2
pairsam
pairsam_px2
r3d
spt
tiff
txt
uncompressed_bowtie2Index
uncompressed_bwaIndex
zip


In [12]:
problematic_w = []

for wf in all_wf:
    #print(wf['display_title'])
    dt = wf['uuid']
    steps = wf.get('steps')
    #1 STEPS
    if not steps:
        print(dt,'NO STEPS')
    for step in steps:
        #2  Name, Inputs, Outputs, Meta
        for step_field in ['name', 'inputs', 'outputs', 'meta']:
            if step_field not in step.keys():
                problematic_w.append(dt)
                print(dt,'NO {} IN STEP {}'.format(step_field, step['name']))
            elif not step[step_field]:
                problematic_w.append(dt)
                print(dt,'NO VALUE FOR {} IN STEP {}'.format(step_field, step['name']))
#         #3 step meta should have software used
#         if step.get('meta'):
#             if 'software_used' not in step['meta'].keys():
#                 print(dt,'NO software_used in IN STEP {} meta'.format(step['name']))
        #4 name meta source in step inputs
        if step.get('inputs'):
            for an_input in step['inputs']:
                for input_field in ['meta', 'source', 'name']:
                    if input_field not in an_input.keys():
                        problematic_w.append(dt)
                        print(dt,'NO {} IN STEP INPUT {}'.format(input_field, step['name']))
                #5 input meta should have type file_format
                if an_input.get('meta'):
                    for input_meta_field in ['type']:
                        if input_meta_field not in an_input['meta'].keys():
                            problematic_w.append(dt)
                            print(dt,'NO {} IN STEP {} INPUT {} META'.format(input_meta_field, step['name'], an_input['name']))
                    if an_input['meta'].get('type'):
                        problematic.append(dt)
                        if an_input['meta']['type'] in ["data file", "reference file"]:
                            if 'file_format' not in an_input['meta'].keys():
                                problematic_w.append(dt)
                                print(dt,'NO file_format IN STEP {} INPUT {} META'.format(step['name'], an_input['name']))

                                # This part is not required anymore since file format is a linkTo
#                             else:
#                                 if an_input['meta']['file_format'] not in allowed_formats:
#                                     print(dt,'NO ALLOWED file_format {} IN STEP {} INPUT {} META'.format(an_input['meta']['file_format'], step['name'], an_input['name']))
                                    
                            
                #6 input source should have name
                if an_input.get('source'):
                    for a_source in an_input['source']:
                        for input_source_field in ['name']:
                            if input_source_field not in a_source.keys():
                                problematic_w.append(dt)
                                print(dt,'NO {} IN STEP INPUT SOURCE {}'.format(input_source_field, step['name']))
        #7 name target in step outputs
        if step.get('outputs'):
            for an_output in step['outputs']:
                for output_field in ['name', 'target']:
                    if output_field not in an_output.keys():
                        problematic_w.append(dt)
                        print(dt,'NO {} IN STEP OUTPUT {}'.format(output_field, step['name']))
                #8 target needs name
                if an_output.get('target'):
                    for a_target in an_output['target']:
                        if 'name' not in a_target.keys():
                            problematic_w.append(dt)
                            print(dt,'NO name IN STEP OUTPUT TARGET {}'.format(step['name']))
    if wf.get('arguments'):
        for arg in wf['arguments']:
            if arg.get('argument_format'):
                if arg['argument_format']['display_title'] not in allowed_formats:
                    problematic_w.append(dt)
                    print(dt,'NO ALLOWED type in argument format {}'.format(arg['argument_format']))
                    
                            
                    
                
# Required properties in “steps.outputs” objects:
# - Name
# - Target
# Required properties in “steps.outputs.target” objects:
# - Name           
# "properties": {
#                                         "type" : {
#                                             "title" : "Input Type",
#                                             "type" : "string",
#                                             "enum": [
#                                                 "data file",
#                                                 "report",
#                                                 "QC",
#                                                 "parameter",
#                                                 "reference file"
#                                             ]

# "file_format": "chromsizes",
# "file_format": "merged_nodups"

4459a4d8-1bd8-4b6a-b2cc-2506f4270a34 NO type IN STEP align INPUT fastq1 META
4459a4d8-1bd8-4b6a-b2cc-2506f4270a34 NO type IN STEP filtersort INPUT input_bam META
4459a4d8-1bd8-4b6a-b2cc-2506f4270a34 NO type IN STEP dedup INPUT input_bam META
bef50397-4d72-4ed1-9c78-100e14e5c47f NO file_format IN STEP Provenance Tracking INPUT inputs META
146da22a-502d-4500-bf57-a7cf0b4b2364 NO type IN STEP align INPUT fastq1 META
146da22a-502d-4500-bf57-a7cf0b4b2364 NO type IN STEP filtersort INPUT input_bam META
146da22a-502d-4500-bf57-a7cf0b4b2364 NO type IN STEP dedup INPUT input_bam META
a9caf6f3-49e5-4c33-bfab-9ec90d65111c NO file_format IN STEP juicer2pairs INPUT input_merged_nodups META
a9caf6f3-49e5-4c33-afab-9ec90d65faf3 NO file_format IN STEP juicer2pairs INPUT input_merged_nodups META
b9829418-49e5-4c33-afab-9ec90d65faf3 NO file_format IN STEP hic2mcool INPUT input_hic META
2324ad76-ff37-4157-8bcc-3ce72b7dace9 NO file_format IN STEP FastQC INPUT limits_file META
2324ad76-ff37-4157-8bcc-3ce72

Required properties in workflow schema:
- steps
Required properties in “steps” objects:
- Name
- Inputs
- Outputs
- Meta
Required properties in “steps.meta” objects:
- software_used
Required properties in “steps.inputs” objects:
- Name
- Meta
- Source
Required properties in “steps.inputs.meta” objects:
- Type
~- file_format~
Required properties in “steps.inputs.source” objects:
- name
Required properties in “steps.outputs” objects:
- Name
- Target
Required properties in “steps.outputs.target” objects:
- Name

In [19]:
problematic_w = list(set(problematic_w))
print(len(problematic_w))
for i in problematic_w:
    print(i)

8
b9829418-49e5-4c33-afab-9ec90d65faf3
c7a60edf-49e5-4c33-afab-9ec90d65faf3
a9caf6f3-49e5-4c33-bfab-9ec90d65111c
4459a4d8-1bd8-4b6a-b2cc-2506f4270a34
146da22a-502d-4500-bf57-a7cf0b4b2364
a9caf6f3-49e5-4c33-afab-9ec90d65faf3
2324ad76-ff37-4157-8bcc-3ce72b7dace9
bef50397-4d72-4ed1-9c78-100e14e5c47f


In [ ]:

8
b9829418-49e5-4c33-afab-9ec90d65faf3
c7a60edf-49e5-4c33-afab-9ec90d65faf3
a9caf6f3-49e5-4c33-bfab-9ec90d65111c
4459a4d8-1bd8-4b6a-b2cc-2506f4270a34
146da22a-502d-4500-bf57-a7cf0b4b2364
a9caf6f3-49e5-4c33-afab-9ec90d65faf3
2324ad76-ff37-4157-8bcc-3ce72b7dace9
bef50397-4d72-4ed1-9c78-100e14e5c47f


ff problematic
a9caf6f3-49e5-4c33-bfab-9ec90d65111c  done
a9caf6f3-49e5-4c33-afab-9ec90d65faf3  done
b9829418-49e5-4c33-afab-9ec90d65faf3  done
2324ad76-ff37-4157-8bcc-3ce72b7dace9  done
c7a60edf-49e5-4c33-afab-9ec90d65faf3  done

In [16]:
import os
folder = 'wf_update'

if not os.path.exists(folder):
        os.makedirs(folder)
        
for i in problematic_w:
    save = ff_utils.get_metadata(i, my_key, add_on = 'frame=raw')
    filename = folder + '/' + i + '.json'
    with open(filename, 'w') as outfile:
        json.dump(save, outfile, indent=4)

In [28]:
import os
folder = 'wf_update'

corrected = ['a9caf6f3-49e5-4c33-bfab-9ec90d65111c',
'a9caf6f3-49e5-4c33-afab-9ec90d65faf3',
'b9829418-49e5-4c33-afab-9ec90d65faf3',
'2324ad76-ff37-4157-8bcc-3ce72b7dace9',
'c7a60edf-49e5-4c33-afab-9ec90d65faf3'
]

# all usable env names
all_envs = ['data', 
            #'fourfront-webdev',
            'fourfront-mastertest']
# get admin key from s3
#
for an_env in all_envs:
    my_key = ff_utils.get_authentication_with_server({}, ff_env = an_env)
    print(an_env)
    for i in corrected:
        filename = folder + '/' + i + '.json'
        patch = json.loads(open(filename).read())
        try:
            temp = ff_utils.get_metadata(patch['uuid'], my_key)
        except Exception as e:
            print(patch['app_name'], patch['uuid'], 'doesnot exist')
            continue
        
        ff_utils.patch_metadata(patch, patch['uuid'], my_key)
        


data
fourfront-mastertest


Exception: Bad status code for PATCH request for http://fourfront-mastertest.9wzadzju3p.us-east-1.elasticbeanstalk.com/2324ad76-ff37-4157-8bcc-3ce72b7dace9: 422. Reason: {'status': 'error', 'code': 422, 'description': 'Failed validation', '@type': ['ValidationFailure', 'Error'], 'title': 'Unprocessable Entity', 'errors': [{'location': 'body', 'name': ['schema_version'], 'description': "request method 'PATCH' is not one of "}, {'location': 'body', 'name': ['category'], 'description': "'QC' is not of type 'array'"}, {'location': 'body', 'name': [], 'description': "Additional properties are not allowed ('data_types', 'workflow_diagram', 'workflow_type' were unexpected)"}]}

In [ ]:
# To Do 

Regrab the corrected from data and patch them to other envs